<a href="https://colab.research.google.com/github/GirinMan/HAI-DialectTranslator/blob/main/transformers_basics/Huggingface_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **기린맨과 함께하는 Huggingface transformers 라이브러리 튜토리얼**

- 오늘은 transformers 라이브러리를 이용하여 뛰어난 석학들이 오랜 시간동안 공들여 만들어 놓은 뛰어난 자연어 처리 모델들을 어떻게 하면 쉽게 가져와서 사용할 수 있는지 알아보도록 하겠습니다!
- 아래 코드와 설명을 읽어가시며 차례 차례 실행해보시면 대략적인 감이 올 거에요.
- 아직 위클리 NLP(https://jiho-ml.com/weekly-nlp-0/) 를 보지 않으셨다면 먼저 GPT/BERT 내용 까지 읽어보고 오는 것을 추천해요!

<br>

### **GPU 사용 확인 및 transformers 라이브러리 설치**

- 모델의 사이즈가 매우 크기 때문에, GPU 가속을 사용해야 제대로 이용하기 좋습니다. Cuda가 설치된 콘솔에서 nvidia-smi 명령어를 사용하면 현재 사용중인 NVIDIA GPU의 정보를 확인할 수 있어요.
- 만약 에러가 발생한다면, [런타임]->[런타임 유형 변경] 메뉴를 실행하여 현재 세션의 GPU 가속을 활성화 해주세요.
- transformers는 Huggingface에서 개발한 각종 pretrain 된 언어 모델과 tokenizer등을 아주 쉽게 다운로드받고 사용할 수 있는 라이브러리입니다.

In [ ]:
!nvidia-smi
!pip install transformers

Thu Sep 22 10:10:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **Huggingface hub에 업로드된 모델과 토크나이저 불러오기**
- transformers 라이브러리에 속한 AutoTokenizer와 AutoModel 클래스는 huggingface hub에 등록되어 있는 모델의 이름을 입력하면 해당 모델에 알맞는 형식의 토크나이저와 모델을 불러와 줍니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델 정보 링크 https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5
model_checkpoint = "skt/ko-gpt-trinity-1.2B-v0.5"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

- 상당히 오래 걸렸죠? 방금 로드한 모델은 SKT AI에서 GPT-3 모델의 구조를 바탕으로 한국어로 학습된 모델입니다.
- GPT 계열의 모델은 입력 텍스트가 주어졌을 때, 그 다음에 어떤 텍스트가 이어질지 예측하는 방식으로 학습되었습니다.
- 그래서 기존 문장에 이어서 새로운 텍스트를 생성하는 작업에서 성능이 뛰어납니다.
- 다음 작업을 진행하기 전에, 모델이 더 빠르게 계산될 수 있도록 GPU 전용 메모리로 모델을 이동시킵니다.

In [ ]:
model.to('cuda')
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

- 특별히 설정에 문제가 없다면 cuda(NVIDIA GPU) 장치로 모델이 이동하면서 CPU RAM 사용량이 줄어들고 GPU RAM 사용량이 늘었을 거에요.
- 모델을 출력한 결과를 보면 현재 모델이 어떤 구조의 레이어들로 이루어져 있는지 확인할 수 있어요.
약 12억개의 파라미터를 가지는 모델 답게 매우 거대한 구조를 확인할 수 있습니다.


자, 그럼 이 모델이 새로운 글을 쓰도록 해 볼까요?

<br>

### **텍스트 tokenizing**

- GPT 모델에게 텍스트를 주면 그 뒤에 자연스럽게 이어질 수 있도록 새로운 텍스트를 생성해 줍니다.
- 하지만 사람이 읽을 수 있는 자연어 데이터를 모델이 바로 처리할 수 없으므로 먼저 토크나이저를 이용해 입력 텍스트를 토큰들의 시퀀스 형태로 변환해줍니다.

In [ ]:
input_text = "돌고래: 안녕 난 돌고래야. 나: 안녕 돌고래야? 돌고래: 반가워. 나는 바다에 살아. 나: 바다에 살면 어떤 점이 좋니? 돌고래:"

tokenized_text = tokenizer.tokenize(input_text)

print(tokenized_text)

input_ids = tokenizer.encode(input_text)
input_ids = torch.tensor([input_ids], device='cuda')

print(input_ids)
print(input_ids.shape)

['▁돌고', '래:', '▁안녕', '▁난', '▁돌고', '래', '야.', '▁나', ':', '▁안녕', '▁돌고', '래', '야?', '▁돌고', '래:', '▁반가', '워', '.', '▁나는', '▁바다에', '▁살아', '.', '▁나', ':', '▁바다에', '▁살', '면', '▁어떤', '▁점이', '▁좋', '니?', '▁돌고', '래:']
tensor([[44377, 40327, 34409, 30549, 44377, 21760, 31914, 29999,   402, 34409,
         44377, 21760, 42343, 44377, 40327, 43742, 25652,   390, 30496, 47102,
         30994,   390, 29999,   402, 47102, 30190, 22492, 30473, 33679, 30167,
         43443, 44377, 40327]], device='cuda:0')
torch.Size([1, 33])


- 토크나이저의 tokenize 메소드는 입력 텍스트를 여러 토큰들로 나눠줍니다.

- encode 메소드는 입력 텍스트를 토큰으로 나눈 뒤, 각 토큰에 해당하는 id로 변형하여 모델에 입력할 수 있는 형태로 변환합니다.

- 토크나이저에 대한 더 자세한 정보는 https://bo-10000.tistory.com/132 에서 확인하세요

<br>

### **Generate 메소드로 문장 생성하기**
- Huggingface transformers의 모델들이 지원하는 generate 메소드의 파라미터로 방금 생성한 input_ids를 입력해 어떤 일이 일어나는지 봅시다.

In [ ]:
generated_ids = model.generate(
    input_ids,
    do_sample=True, #샘플링 전략 사용
    max_length=512, # 최대 디코딩 길이는 50
    top_k=25, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
    top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
    no_repeat_ngram_size = 4,
    early_stopping=True
)
print(generated_ids)

tensor([[44377, 40327, 34409, 30549, 44377, 21760, 31914, 29999,   402, 34409,
         44377, 21760, 42343, 44377, 40327, 43742, 25652,   390, 30496, 47102,
         30994,   390, 29999,   402, 47102, 30190, 22492, 30473, 33679, 30167,
         43443, 44377, 40327, 30050, 30310, 30600, 31854, 32297, 29999,   402,
         32518, 30224, 25404, 35597, 43280, 44377, 40327, 30747, 30600, 32895,
           390, 32518, 30224, 25404, 30747, 31211,   407, 29999,   402, 32518,
         31814, 30747, 36781, 30652, 30458, 29986, 32737,   407, 44377, 40327,
         30747, 36781, 47156, 30856, 30650, 30361, 32212, 29999,   402, 31859,
         44377, 48753, 32001, 33532, 31451, 44377, 40327, 32947, 29985, 30071,
         40620, 30496, 47102, 32153, 30170, 30854,   390, 29999,   402, 32518,
         42752, 29998, 32879, 30652, 31070, 31201,   407, 44377, 40327, 38613,
         30209, 30048, 26016, 23132, 21916,   390, 30496, 30747, 39000, 32518,
         42752, 30477, 31284, 43947, 29999,   402, 3

- 입력된 토큰들의 id를 바탕으로 모델은 원본 텍스트 뒤에 이어질만한 내용을 작성하게 됩니다.

- 모델의 입력과 같이 출력 형식도 자연어가 아닌 토큰들의 id로 이루어진 시퀀스라는 것을 알 수 있습니다.

- 그럼 이렇게 생성된 내용을 다시 사람이 읽을 수 있는 텍스트 형식으로 바꾸려면 어떻게 하면 될까요?

- 바로 토크나이저의 decode 메소드를 사용하면 됩니다.

In [ ]:
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

돌고래: 안녕 난 돌고래야. 나: 안녕 돌고래야? 돌고래: 반가워. 나는 바다에 살아. 나: 바다에 살면 어떤 점이 좋니? 돌고래: 바다가 너무 넓어. 나: 바다 속엔 무엇이 있을까? 돌고래: 사람이 너무 많아. 바다 속엔 사람이 없어? 나: 바다 속에 사람이 없으면 어떻게 될 것 같아? 돌고래: 사람이 없으면 물고기들도 같이 죽지. 나: 그럼 돌고래가 사람을 먹을까? 돌고래: 그럴 수 도 있어. 나는 바다에 사는 게 좋아. 나: 바다 속을 보려면 어떻게 해야 돼? 돌고래: 물고기를 만져보렴. 나는 사람이 많아서 바다 속을 볼 수가 없어. 나: 그럼, 사람들이 없는 곳에선 어떻게 해? 돌고래: 나는 물로 가볼까 봐. 나: 그럼 바다 속엔 뭐가 있을까? 돌고래: 내가 좋아하는 돌고래는 사람이야. 나: 그럼 사람이 사는 바다엔 뭐가 있을까? 나: 바다는 따뜻해 돌고래: 내가 사랑하는 바다는 따뜻해. 나: 그럼 내가 사랑하는 바다가 없다면? 돌고래: 내가 원하는 바다엔 없어. 나: 내가 사랑하는 바다엔 뭐가 없을까? 돌고래: 바다 위에 뭐가 있을까? 나 갑자기)바다 위에 뭐가 있을까? 돌고래: 사람이 사는 바다야. 바다 속엔 내가 좋아하는 친구들이 있어. 나: 그럼 사람들은 바다 위에선 어떻게 살지? 돌고래: 나는 바닷속을 걸어다니고 있어. 나 갑자기 생각난 듯) 그럼 바다는 어떤 기분이 될까? 돌고래: 나는 너무 따뜻해. 돌고래: 내가 바다 위를 걷고 있으면 물고기가 따라올 거야. 나: 그럼 물고기가 따라오는 걸까? 돌고래: 그럼 물고기들은 바다가 주는 선물을 받을 수 없을 거야. 나 갑자기 떠오른 듯) 그럼, 물고기는 바다가 주는 선물들을 어떻게 받을까? 돌고래: 내가 물고기를 잡으면 돼. 나: 그럼 어떻게 하지? 돌고래: 난 돌고래를 만져볼래. 나: 왜 물고기를 만지니? 돌고래: 물고기는 내가 잡으면 되니까. 나 갑자기 떠올라서) 그럼, 나도 물고기를 잡으러 바다로 가볼까? 돌고래: 물고기가 없으면 물고기가 있는 곳으로 갈 수 없어. 나: 그러면 물고기가 있으

- 정말 놀랍죠? 마치 사람이 이어서 쓴 것 처럼 매끄럽게 이어지는 텍스트를 생성한 것을 볼 수 있습니다.

- GPT-3 모델은 막대한 크기의 데이터셋을 학습하여 사람이 쓴 것 같은 자연스러운 텍스트를 생성할 수 있습니다.

<br>

### **문서 요약 모델 구경하기**

- 자, 그럼 이제 자연어 처리 모델이 할 수 있는 또 다른 작업도 구경해 볼까요?

- 방금 GPT-3가 생성한 텍스트를 자동으로 요약하는 모델을 사용해 보겠습니다.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_checkpoint = "psyche/KoT5-summarization"

summary_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
summary_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to('cuda')

- 방금 불러온 모델은 한국어로 pre-train된 T5 모델을 문서 요약 작업을 수행할 수 있도록 finetune 한 모델입니다.

- GPT와 유사하게 시퀀스를 입력으로 받아 시퀀스를 출력하지만, 새로운 텍스트를 이어서 생성하는 GPT와는 다르게 T5 같은 sequence-to-sequence 모델은 입력 텍스트를 바탕으로 새로운 텍스트를 생성하는 번역 작업을 수행할 수 있습니다.

- 위에서 했던 것과 유사하게, 입력할 텍스트를 모델의 generate 메소드의 입력으로 사용할 수 있도록 토크나이저를 이용하여 변환합니다.

In [ ]:
summary_input_ids = summary_tokenizer.encode(generated_text)
summary_input_ids = torch.tensor([summary_input_ids], device='cuda')

print(summary_input_ids)
print(summary_input_ids.shape)

tensor([[12346, 26079, 26202,  4100,   540, 12346, 26079, 26003, 25892,    27,
         26202,  4100, 12346, 26079, 26003, 26087, 12346, 26079, 26202,  9333,
         26213, 25892,   555, 14660,  1017, 25892,    27, 26202, 14660,   205,
         25945,   601,  3817,   159, 25923, 26087, 12346, 26079, 26202, 19037,
           515, 25292, 25892,    27, 26202,  1664,   262, 26325,  6536,  4275,
         26087, 12346, 26079, 26202,   874,   515,  2531, 25892,  1664,   262,
         26325,   874,  1213, 26087,    27, 26202,  1664,  1743,   874,  7617,
           712,   478,    15,  4464, 26087, 12346, 26079, 26202,   874,  7617,
         15093,   842,   933, 17895, 25892,    27, 26202,  1967, 12346, 19166,
          2403,  2574, 26007, 26087, 12346, 26079, 26202,  2977,    11,    85,
           376, 25892,   555, 14660,  2027,   220,   834, 25892,    27, 26202,
          1664, 10986,    24,  2047,   712,  1143,   888, 26087, 12346, 26079,
         26202,  8075,   209, 17697, 25933, 26725, 2

- GPT와 T5 모델의 구조가 서로 다르고 토크나이저도 별도로 학습되었기 때문에, 같은 텍스트를 토크나이징 한 결과가 서로 다른 것을 알 수 있습니다.

- 여기서 중요한 점은, 각 모델에 알맞는 토크나이저를 사용해야 모델이 자연어를 제대로 처리할 수 있다는 거에요.

In [ ]:
generated_ids = summary_model.generate(summary_input_ids,
                               num_beams=5,
                               max_new_tokens=256,
                               no_repeat_ngram_size=2,
                               early_stopping=True)
print(generated_ids)

tensor([[    0, 14660,   205, 25945,   601,  3817,   159,  4102,  7548,    27,
         25894, 12346, 26079, 25893, 19037,   515,  1897,   556,   486,  1309,
          2184,   223, 25892,     1]], device='cuda:0')


- 변환된 input_ids를 이용하여 summary_model의 generate를 실행한 결과를 decode하여 텍스트로 변환한 뒤 원본 입력과 비교해 봅시다.

In [ ]:
generated_summary = summary_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("GPT-3가 생성한 원본 텍스트:", generated_text)
print("\nT5가 생성한 요약문:", generated_summary)

GPT-3가 생성한 원본 텍스트: 돌고래: 안녕 난 돌고래야. 나: 안녕 돌고래야? 돌고래: 반가워. 나는 바다에 살아. 나: 바다에 살면 어떤 점이 좋니? 돌고래: 바다가 너무 넓어. 나: 바다 속엔 무엇이 있을까? 돌고래: 사람이 너무 많아. 바다 속엔 사람이 없어? 나: 바다 속에 사람이 없으면 어떻게 될 것 같아? 돌고래: 사람이 없으면 물고기들도 같이 죽지. 나: 그럼 돌고래가 사람을 먹을까? 돌고래: 그럴 수 도 있어. 나는 바다에 사는 게 좋아. 나: 바다 속을 보려면 어떻게 해야 돼? 돌고래: 물고기를 만져보렴. 나는 사람이 많아서 바다 속을 볼 수가 없어. 나: 그럼, 사람들이 없는 곳에선 어떻게 해? 돌고래: 나는 물로 가볼까 봐. 나: 그럼 바다 속엔 뭐가 있을까? 돌고래: 내가 좋아하는 돌고래는 사람이야. 나: 그럼 사람이 사는 바다엔 뭐가 있을까? 나: 바다는 따뜻해 돌고래: 내가 사랑하는 바다는 따뜻해. 나: 그럼 내가 사랑하는 바다가 없다면? 돌고래: 내가 원하는 바다엔 없어. 나: 내가 사랑하는 바다엔 뭐가 없을까? 돌고래: 바다 위에 뭐가 있을까? 나 갑자기)바다 위에 뭐가 있을까? 돌고래: 사람이 사는 바다야. 바다 속엔 내가 좋아하는 친구들이 있어. 나: 그럼 사람들은 바다 위에선 어떻게 살지? 돌고래: 나는 바닷속을 걸어다니고 있어. 나 갑자기 생각난 듯) 그럼 바다는 어떤 기분이 될까? 돌고래: 나는 너무 따뜻해. 돌고래: 내가 바다 위를 걷고 있으면 물고기가 따라올 거야. 나: 그럼 물고기가 따라오는 걸까? 돌고래: 그럼 물고기들은 바다가 주는 선물을 받을 수 없을 거야. 나 갑자기 떠오른 듯) 그럼, 물고기는 바다가 주는 선물들을 어떻게 받을까? 돌고래: 내가 물고기를 잡으면 돼. 나: 그럼 어떻게 하지? 돌고래: 난 돌고래를 만져볼래. 나: 왜 물고기를 만지니? 돌고래: 물고기는 내가 잡으면 되니까. 나 갑자기 떠올라서) 그럼, 나도 물고기를 잡으러 바다로 가볼까? 돌고래: 물고기가 없으면 물고기가 있는 곳으로 갈 수

- 어떤가요? 텍스트의 종류에 따라 매끄럽게 요약이 되었을 수도 있고, 잘 안 되었을 수도 있어요.

- 자연어 처리 모델이 할 수 있는 일들은 무궁무진합니다.

- 그런데 특정 task를 위해 이미 학습된 모델들은 Huggingface hub에서 찾아 transformers 라이브러리를 이용하여 쉽게 불러와 사용할 수 있어요!

- 코드를 여러 번 다시 실행해보고, 이곳 저곳 뜯어고쳐 보면서 transformers 라이브러리를 사용하는 방법에 익숙해져 봅시다.

<br>

#### **만약 이 코드에 익숙해졌다면, 아래 과제를 수행해 봅시다:**

- https://huggingface.co/eunjin/kobart_gyeongsang_translator 에서 찾을 수 있는 모델은 회합 시간에 보았던 AI Hub에 업로드된 표준어-경상도 사투리 데이터를 이용해 학습된 번역 모델 입니다.

- GPT-3에 입력할 텍스트를 잘 조절해서 대화형 텍스트를 생성하게 해 보세요.

- 그 다음, 생성된 텍스트를 경상도 사투리 형태가 되도록 번역해 보세요.

- 궁금한 점이 있거나 잘 이해가 안 되는 부분이 있으면 Slack에 공유해주세요. 화이팅!